In [2]:
import numpy as np
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister,  transpile
from qiskit.visualization import plot_histogram
 
class GeneralQLBM:
    def __init__(self, dims, velocities, tau, boundary="periodic"):
        self.dims = dims
        self.vels = velocities
        self.Q = len(velocities)
        self.tau = tau
        self.boundary = boundary

        self.pos_qubits = [int(np.log2(n)) for n in dims]
        self.total_pos = sum(self.pos_qubits)
        self.vel_qubits = int(np.ceil(np.log2(self.Q)))

        self.pos_reg = QuantumRegister(self.total_pos, "pos")
        self.vel_reg = QuantumRegister(self.vel_qubits, "vel")
        self.cr = ClassicalRegister(self.total_pos + self.vel_qubits, "c")

        self.qc = QuantumCircuit(self.pos_reg, self.vel_reg, self.cr)
        self._build()

    def _build(self):
        self._state_prep()
        self._streaming()
        self._collision()
        self._boundary()
        self._measure()

    def _state_prep(self):
        self.qc.barrier(label="StatePrep")
        for q in list(self.pos_reg) + list(self.vel_reg):
            self.qc.h(q)

    def _streaming(self):
        self.qc.barrier(label="Streaming")
        for idx, v in enumerate(self.vels):
            ctrl_bits = self._vel_control(idx)
            for axis, shift in enumerate(v):
                if shift == 0:
                    continue
                self._apply_controlled_shift(ctrl_bits, axis, shift)

    def _collision(self):
        self.qc.barrier(label="Collision")
        theta = np.pi / (2 * self.tau)
        for q in self.vel_reg:
            self.qc.ry(theta, q)

    def _boundary(self):
        self.qc.barrier(label="Boundary")
        if self.boundary == "bounce_back":
            if self.vel_qubits == 1 and self.total_pos == 2:
                self.qc.x(self.pos_reg[0]); self.qc.x(self.pos_reg[1])
                self.qc.ccx(self.pos_reg[0], self.pos_reg[1], self.vel_reg[0])
                self.qc.x(self.pos_reg[0]); self.qc.x(self.pos_reg[1])
                self.qc.ccx(self.pos_reg[0], self.pos_reg[1], self.vel_reg[0])

    def _measure(self):
        self.qc.barrier(label="Measure")
        self.qc.measure(self.pos_reg, self.cr[:self.total_pos])
        self.qc.measure(self.vel_reg, self.cr[self.total_pos:])

    def _vel_control(self, idx):
        bits = format(idx, f"0{self.vel_qubits}b")
        return [(self.vel_reg[i], int(b)) for i, b in enumerate(bits)]

    def _apply_controlled_shift(self, ctrl_bits, axis, shift):
        target = sum(self.pos_qubits[:axis])
        for qb, bit in ctrl_bits:
            if bit == 0:
                self.qc.x(qb)
        self.qc.cx(ctrl_bits[-1][0], self.pos_reg[target])
        for qb, bit in ctrl_bits:
            if bit == 0:
                self.qc.x(qb)

    def run(self, shots=512, backend_name=None):
        from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2
        from qiskit.transpiler import generate_preset_pass_manager
        from qiskit.visualization import plot_histogram

        # Initialize IBM Quantum Runtime service
        service = QiskitRuntimeService()

        # Select backend: least busy real device with required qubits (or use backend_name if given)
        if backend_name:
            backend = service.get_backend(backend_name)
        else:
            backend = service.least_busy(simulator=False, operational=True, min_num_qubits=self.total_pos + self.vel_qubits)

        # Generate pass manager for transpilation / optimization
        pm = generate_preset_pass_manager(backend=backend, optimization_level=1)

        # Transpile circuit for backend
        transpiled_qc = transpile(self.qc, backend=backend, optimization_level=3)

        # Create SamplerV2 primitive with pass manager
        sampler = SamplerV2(backend=backend, pass_manager=pm)

        # Run the sampler on a list of circuits (one here)
        job = sampler.run([transpiled_qc], shots=shots)

        # Get the primitive result
        result = job.result()

        # The result is a list of primitive unified block results, one per circuit run
        pub_result = result[0]

        # Extract counts from measurement register "c" (your classical register name)
        counts = pub_result.data.c.get_counts()

        print(f"Counts ({shots} shots): {counts}")
        plot_histogram(counts).show()


gqlbm = GeneralQLBM(
    dims=[16], 
    velocities=[[-1],[0],[1]],  # D1Q3: left, stationary, right
    tau=1.2, 
    boundary="bounce_back"
)

gqlbm.run(shots=1024)

AccountNotFoundError: "Unable to find account. Please make sure an account with the channel name 'ibm_quantum_platform' is saved."